<a href="https://colab.research.google.com/github/bgalerne/IoT_data_science/blob/main/2_iot_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to image processing: Histogram and spatial filtering

In this second notebook we will work on:
1. Histogram manipulation
2. Linear filtering
3. Non linear filtering for image denoising



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.color

import imageio


Downlaod some image files:

In [ ]:
!wget "https://www.idpoisson.fr/galerne/iot_data_science/parrot.bmp"
!wget "https://www.idpoisson.fr/galerne/iot_data_science/piano.png"
!wget "https://www.idpoisson.fr/galerne/iot_data_science/buenosaires3.bmp"
!wget "https://www.idpoisson.fr/galerne/iot_data_science/buenosaires4.bmp"

# Histogram and visualization

## Histogram stretching

Histogram stretching consists in applying an affine contrast change to map the minimal gray-level to 0 and the maximal gray-level to 1 (or 255).

**Exercise:** 
1. Define a function imstretch that given an image u returns the image ustretch = a * (u - b) (with constants a and b
to determine on paper) such that min(ustretch) = 0 and 
max(ustretch) = 1.
2. Apply the function to 


  ```
  u = 0.5 + 10./255.*np.random.randn(256,256)
  ```
  and visualize ```u``` and its stretched version. 

**Recall from Lab session #1:** By default, ```plt.imshow```applies histogram stretching, one needs to use ```.set_clim(0,1)``` to display true gray-levels.




In [ ]:
#TODO:

## Logarithmic contrast change

Linear stretching is used to visualized any data, possibly with negative
values (as we did for noise images, difference between images, etc.).
Sometimes when values have different order of magnitudes, stretching is
not sufficient to visualize data.
Here is an example with the modulus of the discrete Fourier transform of
an image:



In [ ]:
imgrgb = skimage.img_as_float(imageio.imread('parrot.bmp'))
imggray = skimage.color.rgb2gray(imgrgb)
moddft = np.abs(np.fft.fftshift(np.fft.fft2(imggray)))


fig, ax = plt.subplots(1, 2, figsize=(20, 10))
im0 = ax[0].imshow(imggray, cmap=plt.cm.gray)
im0.set_clim(0,1)
ax[0].set_title("Grayscale image")
im1 = ax[1].imshow(moddft, cmap=plt.cm.gray)
ax[1].set_title("Fourier modulus")
fig.tight_layout()
plt.show()


At first inspection the image is black with one pixel at the center...

**Exercise:** 
1. Apply g(x) = log(1+x) to ```mdftu``` and redo the visualization.
2. Explain why at the begining the image was black with one pixel at the
center.

In [ ]:
#TODO:

# Histograms and contrast enhancement

## Input image

In [ ]:
imgrgb = skimage.img_as_float(imageio.imread('parrot.bmp'))
imggray = skimage.color.rgb2gray(imgrgb)
M, N = imggray.shape


fig, ax = plt.subplots(1, 2, figsize=(8, 4))
im0 = ax[0].imshow(imgrgb)
ax[0].set_title("RGB image")
im1 = ax[1].imshow(imggray, cmap=plt.cm.gray)
im1.set_clim(0,1)
ax[1].set_title("Grayscale image")
#plt.colorbar(im1,ax=ax[1])
fig.tight_layout()
plt.show()

## Computing histograms

In the following, we compute and display the gray level histogram and the cumulative histogram of an image. 
 
The cumulative histogram of a discrete image $u$ is an increasing function defined on $\mathbb{R}$ by
$$H_u(\lambda)=\frac{1}{|\Omega|}\#{\{\textbf{x};\;u(\textbf{x})\leq \lambda\}}.$$

We compute the histogram of the image `imgray`

In [ ]:
imhisto, bins = np.histogram(imggray, range=(0,1), bins = 256)
imhisto      = imhisto/np.sum(imhisto)

We now compute the corresponding cumulative histogram thanks to the function ```np.cumsum()``` which cumulates the values of a vector from left to right.

In [ ]:
imhistocum = np.cumsum(imhisto) 

We display the image, histogram and cumulative histogram

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
im0 = axes[0].imshow(imggray,cmap='gray')
im0.set_clim(0,1)
axes[0].set_title('parrot image')
axes[1].bar(np.arange(np.size(imhisto)), imhisto, width=1.)
axes[1].set_title('histogram')
axes[2].bar(np.arange(np.size(imhistocum)), imhistocum, width=1.)
axes[2].set_title('cumulative histogram')
fig.tight_layout()

##Histogram equalization 

If $u$ is a discrete image and $h_u$  its gray level distribution, histogram equalization consists in
applying a contrast change $g$ (increasing function) to $u$ such that $h_{g(u)}$ is as close as possible to a constant distribution. We can compute directly $$H_u(u)*255,$$
where $H_u$ is the cumulative histogram of $u$.

To this aim, we can apply directly the vector `imhistocum` (which can be seen as a function from $\{0,\dots,255\}$ into $[0,1]$) to the numpy array `imgray`. Since `imggray` has values between $0$ and $1$, it is necessary to multiply it by $255$ and cast it as a 8-bit array. 

In [ ]:
imeq = imhistocum[np.uint8(imggray*255)]

### Exercise:
1. Display `imeq` and its associated histogram and cumulative histogram. Comment the result.
2. Code an alternative histogram equalization procedure using a sorting algorithm: Each gray-level of the image having $M\times N$ pixels is replaced by its rank $k\in\{0,...,MN-1\}$  divided by $MN-1$.
3. Check that the results are consistent.
4. Try this algorithm on `piano.png`. What do you observe? Can you explain the artefact?


In [ ]:
#TODO

## Midway histogram

In [ ]:
buenos1 = skimage.img_as_float(imageio.imread('buenosaires3.bmp'))
buenos2 = skimage.img_as_float(imageio.imread('buenosaires4.bmp'))
u = skimage.color.rgb2gray(buenos1)
v = skimage.color.rgb2gray(buenos2)
M, N = u.shape

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
im0 = ax[0].imshow(u, cmap=plt.cm.gray)
im0.set_clim(0,1)
ax[0].set_title("Grayscale image 1")
im1 = ax[1].imshow(v, cmap=plt.cm.gray)
im1.set_clim(0,1)
ax[1].set_title("Grayscale image 2")
fig.tight_layout()
plt.show()

In [ ]:
# sort u and v:
u_sort,index_u=np.sort(u,axis=None),np.argsort(u,axis=None)
v_sort,index_v=np.sort(v,axis=None),np.argsort(v,axis=None)

u_midway = np.zeros(u_sort.shape)
v_midway = np.zeros(v_sort.shape)

u_midway[index_u] = (u_sort + v_sort)/2
v_midway[index_v] = (u_sort + v_sort)/2
u_midway = u_midway.reshape(u.shape)
v_midway = v_midway.reshape(v.shape)

fig, ax = plt.subplots(2, 2, figsize=(12, 12))
im = ax[0,0].imshow(u, cmap=plt.cm.gray)
im.set_clim(0,1)
ax[0,0].set_title("Grayscale image 1")
im = ax[0,1].imshow(v, cmap=plt.cm.gray)
im.set_clim(0,1)
ax[0,1].set_title("Grayscale image 2")
im = ax[1,0].imshow(u_midway, cmap=plt.cm.gray)
im.set_clim(0,1)
ax[1,0].set_title("Midway image 1")
im = ax[1,1].imshow(v_midway, cmap=plt.cm.gray)
im.set_clim(0,1)
ax[1,1].set_title("Midway image 2")
fig.tight_layout()
plt.show()


# Linear filtering

Below we see how to apply a 2D convolution to an image with a small kernel. Note that for kernels with a large support, one should use FFT-based convolution.


## Gaussian blur:

The code below creates a 2D Gaussian kernel.



In [ ]:
import scipy.ndimage

sigblur = 4.
s = int(3*sigblur)
w = 2*s+1
kernel = np.zeros(w)
for t in np.arange(w):
    kernel[t] = np.exp(-(t-s)**2/(2*sigblur**2))
kernel /= sum(kernel)

gausskernel = np.zeros((w,w))
for t1 in np.arange(w):
    for t2 in np.arange(w):
        gausskernel[t1,t2] = kernel[t1]*kernel[t2]
print('Sum of coeff: ', np.sum(gausskernel))
plt.imshow(gausskernel)
plt.colorbar()
plt.show()

imgrgb = skimage.img_as_float(imageio.imread('parrot.bmp'))
imggray = skimage.color.rgb2gray(imgrgb)
M, N = imggray.shape

blurredimg = scipy.ndimage.convolve(imggray, gausskernel)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
im0 = axes[0].imshow(imggray,cmap='gray')
im0.set_clim(0,1)
axes[0].set_title('parrot image')
im1 = axes[1].imshow(blurredimg ,cmap='gray')
im1.set_clim(0,1)
axes[1].set_title('blurred image')
fig.tight_layout()

plt.show()



### Exercise: 

The code above is a lot of lines for such a useful filter. Check the documentation of skimage for the corresponding Gaussian blur function. 

## Derivatives:





In [ ]:

hderker = np.array([[-1.,1.]])
print(hderker.shape)
vderker = hderker.copy().T
print(vderker.shape)

imgrgb = skimage.img_as_float(imageio.imread('parrot.bmp'))
imggray = skimage.color.rgb2gray(imgrgb)
M, N = imggray.shape

hder = scipy.ndimage.convolve(imggray, hderker)
vder = scipy.ndimage.convolve(imggray, vderker)
gradnorm = np.sqrt(hder**2+vder**2)
graddir = np.arctan2(vder,hder)

fig, ax = plt.subplots(2, 2, figsize=(30, 30))
im = ax[0,0].imshow(hder, cmap=plt.cm.gray)
fig.colorbar(im, ax=ax[0,0])
ax[0,0].set_title("Horizontal derivative")
im = ax[0,1].imshow(vder, cmap=plt.cm.gray)
fig.colorbar(im, ax=ax[0,1])
ax[0,1].set_title("Vertical derivative")
im = ax[1,0].imshow(gradnorm, cmap=plt.cm.gray)
im.set_clim(0,1)
fig.colorbar(im, ax=ax[1,0])
ax[1,0].set_title("Gradient norm")
im = ax[1,1].imshow(graddir, cmap=plt.cm.hsv)
im.set_clim(-np.pi,np.pi)
fig.colorbar(im, ax=ax[1,1])
ax[1,1].set_title("Gradient direction")
fig.tight_layout()
plt.show()




### Exercice: Sobel filters and contours:
Let us recall that the Sobel filter uses filters as this one:
$$
        \begin{pmatrix}
          -1 & -2 & -1\\
          0 & 0 & 0\\
          1 & 2 & 1\\
        \end{pmatrix}
$$
to compute directional derivatives.

1. Do a similar figure as above with Sobel filter.
2. Add Gaussian noise to the input image and check that the Sobel filters are less sensitive than the -1/+1 directional derivatives.
3. Apply a threshold to the ```gradnorm```image to extract contours of the image. What is a good threshold value according to you?


# Non linear filtering: Denoising algorithms


## Exercise:

1. Extract a small part of an image that contains details (such as the head of the parrot or the head of the astronaut image). The goal is to see individual pixels when displaying in the notebook.

2. Generate three noisy versions: One with 
    * One with a small amount of Gaussian noise (sigma = 4/255)
    * one with a large amount of Gaussian noise (sigma = 20/255)
    * and one with impulse noise of 5%.

3. Compare the following algorithms one each image:
  * Gaussian blur (used above) with a well-chosen parameter
  * Median filter: ```skimage.filters.median```
  * Bilateral filter: ```skimage.restoration.denoise_bilateral````
  * Non local means.

 For each result display the PSNR by adapting the tutorial here:
https://scikit-image.org/docs/dev/auto_examples/filters/plot_nonlocal_means.html?#non-local-means-denoising-for-preserving-textures



#Credits:
Some parts of this practical session are inspired by: 
* Julie Delon's "Introduction to image processing - Radiometry": 
https://nbviewer.jupyter.org/github/judelo/notebooks/blob/master/python/TP_Radiometrie.ipynb
